In [1]:
import pandas as pd
import numpy as np

In [2]:
icudf = pd.read_csv("https://physionet.org/files/mimiciii-demo/1.4/ICUSTAYS.csv?download")
procdf = pd.read_csv("https://physionet.org/files/mimiciii-demo/1.4/PROCEDUREEVENTS_MV.csv?download")
admitdf = pd.read_csv("https://physionet.org/files/mimiciii-demo/1.4/ADMISSIONS.csv?download")
drugdf = pd.read_csv("https://physionet.org/files/mimiciii-demo/1.4/PRESCRIPTIONS.csv?download")

Identify those who died in the ICU

In [3]:
deaths = admitdf[admitdf['discharge_location'].isin(['DEAD/EXPIRED'])]
icu_admin = pd.merge(icudf, deaths, how='left', on='hadm_id')
icu_admin['discharge_location'].value_counts()

DEAD/EXPIRED    46
Name: discharge_location, dtype: int64

Procedures

In [12]:
procdf = pd.read_csv("https://physionet.org/files/mimiciii-demo/1.4/PROCEDUREEVENTS_MV.csv?download")
topprocs = procdf['ordercategoryname'].value_counts().head(8).index.tolist()
procdf = procdf[procdf['ordercategoryname'].isin(topprocs)].reset_index()
procdf = procdf[procdf['icustay_id'].notna()]
myproc_counts = procdf.groupby(['subject_id', 'icustay_id', 'ordercategoryname']).size().reset_index(name='counts')
myproc_counts_long = myproc_counts.pivot(index = ['subject_id','icustay_id'], 
                                         columns = 'ordercategoryname',
                                         values = 'counts').reset_index()
#.reset_index()
myproc_counts_long = myproc_counts_long.replace(np.nan,0)
#myproc_counts_long = myproc_counts_long.reset_index(drop=True, inplace=True)
myproc_counts_long = myproc_counts_long.rename_axis(None)
myproc_counts_long.head()

ordercategoryname,subject_id,icustay_id,Dialysis,Imaging,Intubation/Extubation,Invasive Lines,Peripheral Lines,Procedures,Significant Events,Ventilation
0,40124,269173,0.0,1.0,1.0,0.0,3.0,6.0,0.0,1.0
1,40124,279554,0.0,0.0,2.0,0.0,3.0,2.0,0.0,1.0
2,40177,236120,0.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0
3,40177,285750,0.0,0.0,1.0,2.0,2.0,0.0,1.0,0.0
4,40204,285369,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0


Drugs

In [10]:
drugdf = pd.read_csv("https://physionet.org/files/mimiciii-demo/1.4/PRESCRIPTIONS.csv?download")
top_twenty = drugdf['formulary_drug_cd'].value_counts().head(20).index.tolist()
#filter rows to only these drugs
drugdf = drugdf[drugdf['formulary_drug_cd'].isin(top_twenty)].reset_index()
drugdf = drugdf[drugdf['icustay_id'].notna()]
#now start making counts based on person and icu_stay_id
mycounts = drugdf.groupby(['subject_id', 'icustay_id', 'formulary_drug_cd']).size().reset_index(name='counts')
mycounts_long = mycounts.pivot(index = ['subject_id','icustay_id'], 
                               columns = 'formulary_drug_cd', values = 'counts').reset_index()
mycounts_long = mycounts_long.replace(np.nan,0)
mycounts_long

formulary_drug_cd,subject_id,icustay_id,ACET325,CALG1I,D5W1000,D5W250,FURO40I,HEPA5I,INSULIN,KCL20P,...,LR1000,MAG2PM,METO25,MORP2I,NACLFLUSH,NS1000,NS250,NS500,VANC1F,VANCOBASE
0,10006,206504.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
1,10013,264446.0,1.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10017,204881.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,10019,228977.0,0.0,3.0,5.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,10026,277021.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,2.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,44083,286428.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,1.0,5.0,0.0,2.0,2.0,4.0,0.0,4.0,0.0,0.0
122,44154,217724.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,...,0.0,2.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0
123,44212,239396.0,1.0,3.0,0.0,11.0,0.0,2.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,3.0,7.0,4.0,7.0,7.0
124,44222,238186.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Merge procedures and drugs to ICU data

In [11]:
df1 = pd.merge(icu_admin, mycounts_long, how='left', on='icustay_id')
analyticdf = pd.merge(icu_admin, myproc_counts_long, how='left', on='icustay_id')
analyticdf

,row_id_x,subject_id_x,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,...,has_chartevents_data,subject_id,Dialysis,Imaging,Intubation/Extubation,Invasive Lines,Peripheral Lines,Procedures,Significant Events,Ventilation
0,12742,10006,142345,206504,carevue,MICU,MICU,52,52,2164-10-23 21:10:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12747,10011,105331,232110,carevue,MICU,MICU,15,15,2126-08-14 22:34:00,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12749,10013,165520,264446,carevue,MICU,MICU,15,15,2125-10-04 23:38:00,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12754,10017,199207,204881,carevue,CCU,CCU,7,7,2149-05-29 18:52:29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12755,10019,177759,228977,carevue,MICU,MICU,15,15,2163-05-14 20:43:56,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,42676,44083,198330,286428,metavision,CCU,CCU,7,7,2112-05-29 02:01:33,...,NaN,44083.0,0.0,0.0,0.0,1.0,2.0,4.0,2.0,0.0
132,42691,44154,174245,217724,metavision,MICU,MICU,50,50,2178-05-14 20:29:55,...,1.0,44154.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,1.0
133,42709,44212,163189,239396,metavision,MICU,MICU,50,50,2123-11-24 14:14:29,...,NaN,44212.0,16.0,14.0,3.0,10.0,3.0,9.0,0.0,1.0
134,42712,44222,192189,238186,metavision,CCU,CCU,7,7,2180-07-19 06:56:38,...,NaN,44222.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0
